In [1]:
import pandas as pd
from tqdm import tqdm

#find all possible major groups in sequence (dataframe)
def find_unique_major_groups_dataframe(sequenceColumn):
    major_grp_list = []
    for ind,i in sequenceColumn.iteritems():
        for j in i:
            if j not in major_grp_list:
                major_grp_list.append(j)
    return major_grp_list

#find all possible major groups in sequence (list)
def find_unique_major_groups(seq):
    major_grp_list = []
    for i in seq:
        for j in i[1]:
            if j not in major_grp_list:
                major_grp_list.append(j)
    return major_grp_list

# find the overall frequency of occurence of major groups (list)
def frequency_bin(seq):
    uniq_sequences = find_unique_major_groups(seq)
    frequency_bin = {x:0 for x in uniq_sequences}
    for i in seq:
        for j in i[1]:
            frequency_bin[j] += 1
    return frequency_bin

# find the overall frequency of occurence of major groups (dataframed)
def frequency_bin_dataframe(sequenceColumn):
    uniq_sequences = find_unique_major_groups_dataframe(sequenceColumn)
    frequency_bin = {x:0 for x in uniq_sequences}
    for ind,i in sequenceColumn.iteritems():
        for j in i:
            frequency_bin[j] += 1
    return frequency_bin

#find prominence of a sequence - a metric to know importance or interest of a sequence
def prominence(sequence):
    sum = 0
    for i in sequence:
        sum += float(freq__['ratio'].loc[freq__['grp'] == i])
    sum = sum * len(sequence)
    return sum

#apply min max norm on the prominence column in the dataframe
def min_max_norm(dataset):
    if isinstance(dataset, list):
        norm_list = list()
        min_value = min(dataset)
        max_value = max(dataset)

        for value in dataset:
            tmp = (value - min_value) / (max_value - min_value)
            norm_list.append(tmp)

    return norm_list


#### CPT_RANGES dataframe

In [7]:
def create_member_cpt_sequences(input_sas_dataframe):
    df = pd.read_sas(input_sas_dataframe,format='sas7bdat',encoding='iso-8859-1')
    cpt_groups_df =  pd.read_pickle(r'cpt_ranges.pkl')
    sub_group_map = pd.concat([pd.Series(row['SUB_GR'], row['CPT_RANGES']) 
           for _, row in cpt_groups_df.iterrows()]).reset_index()

    sub_group_map.columns = ['CPT','SUB_GR']
    sub_grouped = pd.merge(df, sub_group_map, how='left',left_on='CPT_PROC_CD', right_on='CPT')
    sub_grouped = sub_grouped.dropna(subset=['SUB_GR','CPT'])
    sub_grouped['SUB_GR'] = sub_grouped['SUB_GR'].astype(int)
    member_cpt_sequences = sub_grouped.groupby(['MBR_ID'])['SUB_GR'].apply(list).reset_index(name='CPT_SEQUENCES')
    
    return member_cpt_sequences

In [20]:
def mine_sequence_patterns(sequence_dataframe):
    from prefixspan import PrefixSpan
    db = sequence_dataframe['CPT_SEQUENCES']
    ps = PrefixSpan(db)
    sequences = ps.topk(100,closed=True)
    return sequences

In [55]:
def process_odds_ratio(the_dataframe_greater, the_dataframe_lower):
    the_dataframe_greater['sequence'] = the_dataframe_greater['sequence'].astype(str)
    the_dataframe_lower['sequence'] = the_dataframe_lower['sequence'].astype(str)
    df_greater_ = the_dataframe_greater
    df_lower_ = the_dataframe_lower
    d = {}
    for i in df_greater_.iterrows():
        freq = i[1][0]
        seq  = i[1][1]
        for j in df_lower_.iterrows():
            freq_ = j[1][0]
            seq_ = j[1][1]
            if seq == seq_:
                diff = freq-freq_
                word = str(seq)
                d[word] = 0
                a = freq
                b = freq_
                d[word] =  a/b
                continue
    import ast
    klist = []
    for key,value in d.items():
        res = ast.literal_eval(key)
        klist.append((res,value))

    return klist


In [99]:
def seq_in_seq(sub, full):
        f = ''.join([repr(d) for d in full]).replace("'", "")
        s = ''.join([repr(d) for d in sub]).replace("'", "")
        #return f.find(s) #<-- not reliable for finding indices in all cases
        return s in f

In [100]:
import ast
def ret_list(seq_):
    res = ast.literal_eval(seq_)
    return res

In [8]:
member_cpt_sequences = create_member_cpt_sequences(r'mbr_id_above400.sas7bdat')

In [104]:
above_0 = member_cpt_sequences


In [66]:
above_0_seq = mine_sequence_patterns(above_0)

In [72]:
above_0_seq_df = pd.DataFrame(seq,columns=['frequency','sequence'])

In [68]:
#this is only for demo. I am taking arbitrary set but in production this should be the made from the sas table of the members 
# who have net savings = 0
below_1_seq = member_cpt_sequences[:30]

In [69]:
below_1_seq_ = mine_sequence_patterns(k)
below_1_seq_df_ = pd.DataFrame(below_1_seq_,columns=['frequency','sequence'])

In [73]:
d = process_odds_ratio(above_0_seq_df , below_1_seq_df_)
finalframe = pd.DataFrame(d,columns=['sequence','odds'])

In [74]:
finalframe['odds_min_max_normed'] = min_max_norm(list(frame['odds']))

In [75]:
finalframe

,sequence,odds,odds_min_max_normed
0,[101],0.416667,1.000000
1,[46],0.727273,0.880682
2,[55],0.666667,0.568182
3,"[101, 101]",0.444444,0.284091
4,"[46, 101]",0.875000,0.454545
5,"[55, 46]",0.777778,0.681818
6,"[55, 55]",0.700000,0.184659
7,"[55, 101]",0.777778,0.340909
8,[56],0.636364,0.227273
9,[68],0.538462,0.416667


In [76]:
validation_df = pd.read_pickle(r'finalvalseq')

In [77]:
validation_df.head()

,frequency,sequence
0,2927,[101]
1,2015,"[101, 101]"
2,1518,[154]
3,1478,"[101, 101, 101]"
4,1268,"[154, 154]"


In [78]:
finalframe['sequence'] = finalframe['sequence'].astype(str)
validation_df['sequence'] = validation_df['sequence'].astype(str)

In [89]:
validation_df_join = pd.merge(finalframe, validation_df, how="inner",left_on="sequence",right_on='sequence').drop('frequency', axis=1)

In [90]:
validation_df_join.sort_values("odds_min_max_normed", axis = 0, ascending = False, inplace = True, na_position ='last') 

In [91]:
validation_df_join

,sequence,odds,odds_min_max_normed
0,[101],0.416667,1.000000
1,[46],0.727273,0.880682
11,"[55, 46, 101]",0.857143,0.681818
5,"[55, 46]",0.777778,0.681818
2,[55],0.666667,0.568182
15,"[55, 55, 101, 101]",0.857143,0.522727
28,"[101, 154, 154]",0.571429,0.482955
4,"[46, 101]",0.875000,0.454545
9,[24],0.750000,0.416667
10,"[46, 46]",0.750000,0.416667


In [101]:
k = []
for i in validation_df_join['sequence']:
    seq_ = ret_list(i)
    for ind in member_cpt_sequences.index:
        if seq_in_seq(seq_, member_cpt_sequences['CPT_SEQUENCES'][ind]):
            d = {}
            d['member_id'] = member_cpt_sequences['MBR_ID'][ind]
            d['CPT_SEQ'] = member_cpt_sequences['CPT_SEQUENCES'][ind]
            d['seq_'] = seq_
            k.append(d)

In [102]:
member_ids = []
for member in k:
    k = member['member_id']
    member_ids.append(k)
    
